# Imports

In [2]:
from sklearn.linear_model import LogisticRegression
import torch
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# Constants

In [3]:
DATASET = "com2sense"
MODEL_NAME = "gemma-2-2b"
DATA_DIR = f"./experimental_data/{MODEL_NAME}/{DATASET}/"
WEIGHTS_DIR = f"./weights/linear_analysis/{MODEL_NAME}/{DATASET}/"
TRAIN_SIZE = 0.8
TOP_K = 10  # Set to -1 to see every element

# Data

Load data

In [4]:
acts_exp_resid = torch.load(DATA_DIR + "acts_exp_resid.pt")
acts_resid = torch.load(DATA_DIR + "acts_resid.pt")

C:\Users\wsr20\AppData\Local\Temp\ipykernel_2968\2989447155.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  acts_exp_resid = torch.load(DATA_DIR + "acts_exp_resid.pt")
C

In [5]:
print(acts_exp_resid.shape)

torch.Size([1874, 26, 2304])


Split data

In [6]:
X_train_index = int(TRAIN_SIZE * acts_exp_resid.shape[0])
X_train_exp_resid = acts_exp_resid[:X_train_index, :, :]
X_train_resid = acts_resid[:X_train_index, :, :]
X_test_exp_resid = acts_exp_resid[X_train_index:, :, :]
X_test_resid = acts_resid[X_train_index:, :, :]

In [7]:
print(X_train_resid.shape, X_test_resid.shape)

torch.Size([1499, 26, 2304]) torch.Size([375, 26, 2304])


1 indicates CoT, 0 is Non-CoT

In [8]:
y_train_exp_resid = torch.ones(X_train_exp_resid.shape[0])
y_train_resid = torch.zeros(X_train_resid.shape[0])
y_test_exp_resid = torch.ones(X_test_exp_resid.shape[0])
y_test_resid = torch.zeros(X_test_resid.shape[0])

In [9]:
print(y_train_resid.shape, y_test_resid.shape)

torch.Size([1499]) torch.Size([375])


Concatenate the data

In [10]:
X_train = torch.cat((X_train_exp_resid, X_train_resid), dim=0)
X_test = torch.cat((X_test_exp_resid, X_test_resid), dim=0)
y_train = torch.cat((y_train_exp_resid, y_train_resid), dim=0)
y_test = torch.cat((y_test_exp_resid, y_test_resid), dim=0)

In [11]:
print(X_train.shape, y_train.shape)

torch.Size([2998, 26, 2304]) torch.Size([2998])


# Initialize and train classifiers

In [12]:
classifiers = [LogisticRegression(fit_intercept=False)] * acts_resid.shape[1]

In [13]:
X_train, y_train, X_test, y_test = X_train.cpu().numpy(), y_train.cpu().numpy(), X_test.cpu().numpy(), y_test.cpu().numpy()
for i, classifier in enumerate(classifiers):
    classifier.fit(X_train[:, i, :], y_train)

# Evaluate classifiers

In [14]:
weights_list = []
for i, classifier in enumerate(classifiers):
    y_pred = classifier.predict(X_test[:, i, :])
    print(f"layer {i}")
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:\n", conf_matrix)
    
    class_report = classification_report(y_test, y_pred)
    print("Classification Report:\n", class_report)

    weights = classifier.coef_
    print(f"Weights: {weights}")
    weights_list.append(weights)

    top_k = np.argsort(weights[0, :])[::-1][:TOP_K]
    print(f"Top k indeces: {top_k}")

    print("\n")

layer 0
Accuracy: 0.5026666666666667
Confusion Matrix:
 [[  2 373]
 [  0 375]]
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      0.01      0.01       375
         1.0       0.50      1.00      0.67       375

    accuracy                           0.50       750
   macro avg       0.75      0.50      0.34       750
weighted avg       0.75      0.50      0.34       750

Weights: [[ 0.00319615 -0.00143483 -0.00128562 ... -0.00271136  0.00181736
  -0.00137571]]
Top k indeces: [1393  689 2269 1227 1824  714 1413 2257  331 1788]


layer 1
Accuracy: 0.5053333333333333
Confusion Matrix:
 [[  4 371]
 [  0 375]]
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      0.01      0.02       375
         1.0       0.50      1.00      0.67       375

    accuracy                           0.51       750
   macro avg       0.75      0.51      0.35       750
weighted avg       0.75      0.51  

Starting from layer 7 we are able to obtain perfect prediction. However, it seems like certain neurons are always activated.

# Save weights

In [15]:
for i, weights in enumerate(weights_list):
    weights = torch.tensor(weights)
    torch.save(weights, WEIGHTS_DIR + f"layer_{i}")

RuntimeError: Parent directory ./weights/linear_analysis/gemma-2-2b/com2sense does not exist.